In [1]:
import numpy as np
import pandas as pd
import pyreadr
import os

# Overview
This code cleans the paired comparison data from shiny2 and shiny3 from this [paper](https://gking.harvard.edu/publications/how-measure-legislative-district-compactness-if-you-only-know-it-when-you-see-it). In particular, it creates a new csv called `paired_comparisons.csv` whose columns are 

`['id', 'study', 'district1', 'district2', 'winner', 'alternate_id_1', 'alternate_id_2', 'alternate_id_winner']`
where 

* `id` = anonymized id of person answering questions. Note, id `hyF6TZwPOEVS` was used as an id for 16 respondants in shiny3 though Aaron says this is truly from 16 different people.
* `study` = shiny2 or shiny3 indicating which study the data was from
* `district1` and `district2` are the two districts shown in the pairwise comparison question in the format of `FIPS_[0 if lower chamber, 1 if upper chamber]_districtnumber.jpg`
* `winner` = the district out of the two that was chosen as *more* compact
* `alternate_id_1`, `alternate_id_2`, `alternate_id_winner` = `FIPS_[L/U/C for lower, upper, congressional]_districtnumber_year`. We need this alternate id in order to get the features for each district based on `preds_6_14_18.RData`. The value in this column will be equal to the value in the `disctrict` column in `preds_6_14_18.RData`. I decided to make this a separate column since it was not straight forward to translate the old naming convention to the new naming convention as discussed later.

You will need the following folders and package though you will have to change the directories of these folders in the code because the code depends on my directories:
* `grey maps`
* `paired_comparisons/shiny_results2/`
* `paired_comparisons/shiny_results3/`
* `preds_6_14_18.RData`
* `pyreadr` (library for extracting `.RData` data into python)
    
See the cells below for information about each study, i.e. how many people, how many questions each person answered, and how many unique districts showed up in the questions. There is no overlap between the districts used in shiny2 vs shiny3.

In [2]:
# create new dataframe
col_names =  ['id', 'study', 'district1', 'district2', 'winner', 'alternate_id_1', 'alternate_id_2', 'alternate_id_winner']
final_df  = pd.DataFrame(columns = col_names)

# shiny2
298 people on mturk answered Mturk answered 12 paired comparison questions for 88 districts. 
They were asked to pick the more compact district between the two (Aaron verified this).

Also, Aaron said to check whether (1) the first X rows correspond to the first item shown over the X questions and the next X rows correspond to the second items in each question or (2) if the first row corresponds to the first item in the first question, and the second row correseponds to the second item in the first question, and so on. It appears to be the former based on some manual checking but also there was a file whose first two rows was the same district. I assume they would not ask the same district in a pairwise comparison. Aaron said whatever convention was used should be consistent from study to study.

In [3]:
def extract_file_name(data):
    idx = data[::-1].index('/')
    return data[len(data) - idx :]

In [4]:
# grab all of the file names with the paired comparison data
result_files = [filename for filename in os.listdir('paired_comparisons/shiny_results2/') if filename.startswith("results")]

print('There are', len(result_files), 'people in the study.')

districts = []
num_questions = []

# read in each file one at a time
for f in result_files:
    df = pd.read_csv('paired_comparisons/shiny_results2/' + f, names=["num", "data"])
    df.rename( columns={'Unnamed: 0':'num', 'x':'data'}, inplace=True)
    
    name = ''
    study = 'shiny2'
    district1 = []
    district2 = []
    winner = []
    
    # go through the data in one file
    for index, row in df.iterrows():
        if row['num'] == 1:
            name = row['data']
        if row['num'] > 1 and row['data'] != 'District 1' and row['data'] != 'District 2': 
            if row['num'] <= 13:
                districts.append(extract_file_name(row['data']))
                district1.append(extract_file_name(row['data']))
            elif row['num'] > 13:
                districts.append(extract_file_name(row['data']))
                district2.append(extract_file_name(row['data']))
        elif row['data'] == 'District 1':
            winner.append(district1[len(winner)])
        elif row['data'] == 'District 2':
            winner.append(district2[len(winner)])
    
    # double check that all the files have the same length
    num_questions.append(len(df))
    
    # insert into new data frame
    for i in range(len(district1)):
        
        final_df.loc[len(final_df)] = [name,study, district1[i], district2[i], winner[i], 0, 0, 0]
    
print('There are', len(set(districts)), 'different districts used in the comparisons.')
print('Length of file for each person', set(num_questions))

There are 298 people in the study.
There are 88 different districts used in the comparisons.
Length of file for each person {38}


# shiny3
90 people on mturk answered Mturk answered 20 paired comparison questions for 34 districts. They were asked to pick the most compact district between the two.

In [5]:
# grab all of the file names with the paired comparison data
result_files = [filename for filename in os.listdir('paired_comparisons/shiny_results3/') if filename.startswith("results")]

print('There are', len(result_files), 'people in the study.')

districts2 = []
num_questions = []

# read in each file one at a time
for f in result_files:
    df = pd.read_csv('paired_comparisons/shiny_results3/' + f, names=["num", "data"])
    df.rename( columns={'Unnamed: 0':'num', 'x':'data'}, inplace=True)
    
    name = ''
    study = 'shiny3'
    district1 = []
    district2 = []
    winner = []
    
    # go through the data in one file
    for index, row in df.iterrows():
        if row['num'] == 1:
            name = row['data']
            if name == 'hyF6TZwPOEVS':
                name = f[9:len(f)-4]
        if row['num'] > 1 and row['data'] != 'District 1' and row['data'] != 'District 2': 
            if row['num'] <= 21:
                districts2.append(extract_file_name(row['data']))
                district1.append(extract_file_name(row['data']))
            elif row['num'] > 21:
                districts2.append(extract_file_name(row['data']))
                district2.append(extract_file_name(row['data']))
        elif row['data'] == 'District 1':
            winner.append(district1[len(winner)])
        elif row['data'] == 'District 2':
            winner.append(district2[len(winner)])
    
    # double check that all the files have the same length
    num_questions.append(len(df))

    # insert into new data frame
    for i in range(len(district1)):
        final_df.loc[len(final_df)] = [name,study, district1[i], district2[i], winner[i], 0, 0, 0]
    
print('There are', len(set(districts)), 'different districts used in the comparisons.')
print('Length of file for each person', set(num_questions))

There are 90 people in the study.
There are 88 different districts used in the comparisons.
Length of file for each person {62}


There is no overlap between the districts used in shiny2 vs shiny3.

In [6]:
districts = set(districts)
districts2 = set(districts2)
print(districts.intersection(districts2))

set()


Let's take a peak at the dataframe to make sure all looks good so far.

In [7]:
final_df

,id,study,district1,district2,winner,alternate_id_1,alternate_id_2,alternate_id_winner
0,eMqADFqLbqRD,shiny2,30_1_001.jpg,48_0_006.jpg,48_0_006.jpg,0,0,0
1,eMqADFqLbqRD,shiny2,40_0_074.jpg,48_0_006.jpg,48_0_006.jpg,0,0,0
2,eMqADFqLbqRD,shiny2,38_0_041.jpg,40_0_074.jpg,38_0_041.jpg,0,0,0
3,eMqADFqLbqRD,shiny2,48_0_006.jpg,40_0_074.jpg,40_0_074.jpg,0,0,0
4,eMqADFqLbqRD,shiny2,40_0_074.jpg,38_0_041.jpg,38_0_041.jpg,0,0,0
5,eMqADFqLbqRD,shiny2,48_0_006.jpg,38_0_041.jpg,38_0_041.jpg,0,0,0
6,eMqADFqLbqRD,shiny2,38_0_041.jpg,30_1_001.jpg,38_0_041.jpg,0,0,0
7,eMqADFqLbqRD,shiny2,30_1_001.jpg,38_0_041.jpg,38_0_041.jpg,0,0,0
8,eMqADFqLbqRD,shiny2,30_1_001.jpg,40_0_074.jpg,40_0_074.jpg,0,0,0
9,eMqADFqLbqRD,shiny2,40_0_074.jpg,30_1_001.jpg,40_0_074.jpg,0,0,0


Now we need to translate the old naming convention `FIPS_[0 if lower chamber, 1 if upper chamber]_districtnumber.jpg` that appeared in the shiny2 and shiny3 results to the new naming convention of `FIPS_[L/U/C for lower, upper, congressional]_districtnumber_year` so that we can get features for each of the districts found in `preds_6_14_18.RData`. Some districts have more than one year associated to it in the `preds_6_14_18.RData` file. 


Unfortunately, for each district shown in the paired comparison, it was not recorded which year was used. This is an issue since the same district from different years can have widley different features. Aaron said "[the years] will be from the chambers of the state legislature (rather than Congress) from either 2000-2010 or 2010-2020 election cycles." He suggested to match up the district names in shiny2 and shiny3 to file names in `grey maps`, which is a folder of district images. 

Each district in shiny2 and shiny3 matched to at least one file `grey maps`. However, in some cases there were two matches, i.e. there was an image of that district for two different years. Fortunately, only one of the years existed in `preds_6_14_18.RData`, so hopefully this is reliable. My only hesitation to all of this is that I noticed in `grey_maps`, there were slightly different file names by appending a `0` to parts of the file name for what I would assume describes the same district though based on the actual images this sort of does not seem to be the case. For instance, there are two files called `01_L_14_2000.jpg` and `01_L_014_2000.jpg` that differ by a `0` and the images of these two files are actually quite different, though similar.


In [8]:
#read in feature data for all the districts
result = pyreadr.read_r('preds_6_14_18.RData') 
print(result.keys()) 
df_features = result["finalpreds"]

odict_keys(['finalpreds'])


In [9]:
#get all the file names in grey maps
files = os.listdir('../../Dropbox/grey maps/')

In [10]:
# get each district that appeared in shiny2 and shiny3
districts = list(final_df['district1'].values)
districts.extend(list(final_df['district2'].values))
districts = set(districts)

In [11]:
'''
this function
1. translates the 0 from the old naming convention to L for the new naming convention, and similarly does the same for 1/U 
2. removes the .jpg part of the name
3. removes a leading 0 since none of the district names in `preds_6_14_18` started with a 0

'''
def rename(data):
    num_to_char = {0:'L', 1:'U'}
    indices = [i for i, a in enumerate(data) if a == '_']
    num = data[indices[0] + 1 : indices[1]]
    new = data[: indices[0] + 1] + num_to_char[int(num)] + data[indices[1] : data.index('.')]
    if new[0] == '0':
        new = new[1:]
    return new



# this function checks if a district exists in grey maps and also exists in preds_6_14_18
def exists(data, files, df):
    matched = []
    for f in files:
        if data in f and data[0:2] == f[0:2]:
            matched.append(f)
            
    for m in matched:
        if not df.isin([m[:-4]]).any()[0]:
            matched.remove(m)
        
    return matched

In [12]:
old_name_to_new_name = {}
for f in districts:
    result = exists(rename(f), files, df_features)
    old_name_to_new_name[f] = result

Yay, each district from the shiny2 and shiny3 study matches up to a unique district in `preds_6_14_18` by going through `grey maps`

In [13]:
old_name_to_new_name

{'50_0_B3.jpg': ['50_L_B3_2010.jpg'],
 '40_0_074.jpg': ['40_L_074_2010.jpg'],
 '15_0_050.jpg': ['15_L_050_2010.jpg'],
 '25_0_107.jpg': ['25_L_107_2010.jpg'],
 '09_0_014.jpg': ['9_L_014_2010.jpg'],
 '46_1_022.jpg': ['46_U_022_2010.jpg'],
 '04_0_022.jpg': ['4_L_022_2010.jpg'],
 '24_0_017.jpg': ['24_L_017_2010.jpg'],
 '13_0_170.jpg': ['13_L_170_2010.jpg'],
 '34_0_013.jpg': ['34_L_013_2010.jpg'],
 '25_0_198.jpg': ['25_L_198_2010.jpg'],
 '26_0_081.jpg': ['26_L_081_2010.jpg'],
 '17_0_020.jpg': ['17_L_020_2010.jpg'],
 '34_0_015.jpg': ['34_L_015_2010.jpg'],
 '20_1_039.jpg': ['20_U_039_2010.jpg'],
 '32_0_031.jpg': ['32_L_031_2010.jpg'],
 '09_0_012.jpg': ['9_L_012_2010.jpg'],
 '30_1_037.jpg': ['30_U_037_2010.jpg'],
 '44_0_044.jpg': ['44_L_044_2010.jpg'],
 '31_1_028.jpg': ['31_U_028_2010.jpg'],
 '26_0_038.jpg': ['26_L_038_2010.jpg'],
 '05_1_017.jpg': ['5_U_017_2010.jpg'],
 '20_0_031.jpg': ['20_L_031_2010.jpg'],
 '38_0_041.jpg': ['38_L_041_2010.jpg'],
 '56_0_001.jpg': ['56_L_001_2010.jpg'],
 '27_1

Now time to record the new naming convention in `final_df`

In [14]:
for name in old_name_to_new_name:
    new_name = old_name_to_new_name[name]
    final_df.loc[final_df.district1 == name, 'alternate_id_1'] = new_name
    final_df.loc[final_df.district2 == name, 'alternate_id_2'] = new_name
    final_df.loc[final_df.district2 == name, 'alternate_id_winner'] = new_name

In [15]:
final_df

,id,study,district1,district2,winner,alternate_id_1,alternate_id_2,alternate_id_winner
0,eMqADFqLbqRD,shiny2,30_1_001.jpg,48_0_006.jpg,48_0_006.jpg,30_U_001_2010.jpg,48_L_006_2010.jpg,48_L_006_2010.jpg
1,eMqADFqLbqRD,shiny2,40_0_074.jpg,48_0_006.jpg,48_0_006.jpg,40_L_074_2010.jpg,48_L_006_2010.jpg,48_L_006_2010.jpg
2,eMqADFqLbqRD,shiny2,38_0_041.jpg,40_0_074.jpg,38_0_041.jpg,38_L_041_2010.jpg,40_L_074_2010.jpg,40_L_074_2010.jpg
3,eMqADFqLbqRD,shiny2,48_0_006.jpg,40_0_074.jpg,40_0_074.jpg,48_L_006_2010.jpg,40_L_074_2010.jpg,40_L_074_2010.jpg
4,eMqADFqLbqRD,shiny2,40_0_074.jpg,38_0_041.jpg,38_0_041.jpg,40_L_074_2010.jpg,38_L_041_2010.jpg,38_L_041_2010.jpg
5,eMqADFqLbqRD,shiny2,48_0_006.jpg,38_0_041.jpg,38_0_041.jpg,48_L_006_2010.jpg,38_L_041_2010.jpg,38_L_041_2010.jpg
6,eMqADFqLbqRD,shiny2,38_0_041.jpg,30_1_001.jpg,38_0_041.jpg,38_L_041_2010.jpg,30_U_001_2010.jpg,30_U_001_2010.jpg
7,eMqADFqLbqRD,shiny2,30_1_001.jpg,38_0_041.jpg,38_0_041.jpg,30_U_001_2010.jpg,38_L_041_2010.jpg,38_L_041_2010.jpg
8,eMqADFqLbqRD,shiny2,30_1_001.jpg,40_0_074.jpg,40_0_074.jpg,30_U_001_2010.jpg,40_L_074_2010.jpg,40_L_074_2010.jpg
9,eMqADFqLbqRD,shiny2,40_0_074.jpg,30_1_001.jpg,40_0_074.jpg,40_L_074_2010.jpg,30_U_001_2010.jpg,30_U_001_2010.jpg


In [16]:
final_df.to_csv('paired_comparisons.csv')